<a href="https://colab.research.google.com/github/Puneet-Bajaj-IITM/01245/blob/main/Chat_Summarizer_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl https://ollama.ai/install.sh | sh
!nohup ollama serve &
!pip install ollama
!ollama pull llama2
!pip install flask-ngrok
!pip install pyngrok

import pandas as pd
import concurrent.futures

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10044    0 10044    0     0  34472      0 --:--:-- --:--:-- --:--:-- 34515
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/

In [2]:
import os

templates_folder = '/content/templates'
if not os.path.exists(templates_folder):
    os.makedirs(templates_folder)

index_content = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>File Upload and Processing</title>
    <!-- Bootstrap CSS -->
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha1/dist/css/bootstrap.min.css" rel="stylesheet">
    <!-- Tailwind CSS -->
    <link href="https://cdn.jsdelivr.net/npm/tailwindcss@3.0.7/dist/tailwind.min.css" rel="stylesheet">
    <!-- Custom CSS -->
    <style>
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        .spinner {
            display: none;
        }
        .output-section {
            display: none;
        }
        .rolling-animation {
            border: 6px solid #3498db; /* Blue color */
            border-top: 6px solid transparent; /* Transparent border on top */
            border-radius: 50%; /* Make it a circle */
            width: 40px;
            height: 40px;
            animation: spin 1s linear infinite; /* Apply the spin animation */
        }
    </style>
</head>
<body class="bg-gray-100 p-4">
    <div class="container mx-auto">
        <h1 class="text-3xl font-bold mb-4">Upload Excel Files</h1>
        <form action="/upload" method="post" enctype="multipart/form-data" class="mb-4" onsubmit="return validateForm()">
            <div class="mb-3">
                <label for="files" class="form-label">Choose Excel Files:</label>
                <input type="file" class="form-control" id="files" name="files[]" multiple accept=".xlsx" onchange="displaySelectedFiles(this)">
            </div>
            <button type="submit" class="btn btn-primary" id="uploadBtn">Upload</button>
            <p id="error" style="color: red; display: none;">Please select at least one file.</p>
        </form>
        <div class="spinner" id="spinner">
            <div class="rolling-animation"></div> <!-- Blue rolling animation -->
        </div>
        <h2 class="text-2xl font-bold mb-2">Selected Files:</h2>
        <ul class="list-disc pl-5" id="selectedFilesList">
            <!-- Selected files will be displayed here -->
        </ul>
        {% if output_files %}
        <div class="output-section" id="outputSection">
            <h2 class="text-2xl font-bold mb-2">Processed Files:</h2>
            <ul class="list-disc pl-5" id="outputFilesList">
                <!-- Output files will be displayed here -->
                {% for file in output_files %}
                <li>{{ file }}</li>
                {% endfor %}
            </ul>
        </div>
        {% endif %}

    </div>
    <script>
        function displaySelectedFiles(input) {
            const fileList = document.getElementById('selectedFilesList');
            fileList.innerHTML = ''; // Clear previous list

            const files = input.files;
            for (let i = 0; i < files.length; i++) {
                const listItem = document.createElement('li');
                listItem.textContent = files[i].name;
                fileList.appendChild(listItem);
            }
        }

        function validateForm() {
            const files = document.getElementById('files').files;
            if (files.length === 0) {
                // No files selected, display error message and prevent form submission
                document.getElementById('error').style.display = 'block';
                return false;
            }
            // Show spinner while waiting for output
            document.getElementById('spinner').style.display = 'block';
            return true; // Proceed with form submission
        }

        // Example function to simulate output files appearing after 3 seconds
        setTimeout(() => {
            document.getElementById('spinner').style.display = 'none';
            document.getElementById('outputSection').style.display = 'block';
        }, 3000);
    </script>
</body>
</html>


"""

with open(os.path.join(templates_folder, 'index.html'), 'w') as f:
    f.write(index_content)

print("Templates folder and index.html created successfully.")

Templates folder and index.html created successfully.


In [3]:

def generate_summary(row):
    if row['Chat']:
        res = ollama.generate(model='llama2', prompt=f"""Summarize {row['Chat']}, mention the main issues encountered by the customer, the solutions provided by agent and overarching themes discussed""")
    return res['response'] or None

def process_row(row):
    summary = generate_summary(row)
    return summary

def process_file(filename):
    data = pd.read_excel(filename, header=None)
    data['Summary'] = None
    data.columns = ['Chat', 'Summary']

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_row, row) for _, row in data.iterrows()]
        for future, (index, _) in zip(futures, data.iterrows()):
            summary = future.result()
            data.at[index, 'Summary'] = summary
    return data


In [ ]:
from flask import Flask, request, render_template, send_file
from pyngrok import ngrok
import ollama
import os
import pandas as pd
ngrok.set_auth_token("2aGUSiqfIzgKx0yh6XHaTX3G11f_5HRDp7atGGPtyxaiH91bc")

app = Flask(__name__)

UPLOAD_FOLDER = '/content/uploads'
OUTPUT_FOLDER = '/content/outputs'
ALLOWED_EXTENSIONS = {'xlsx'}

if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['OUTPUT_FOLDER'] = OUTPUT_FOLDER

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def index():
    output_files = os.listdir(app.config['OUTPUT_FOLDER'])
    return render_template('index.html', output_files=output_files)

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'files[]' not in request.files:
        return 'No file part'

    files = request.files.getlist('files[]')
    for file in files:
        if file and allowed_file(file.filename):
            filename = file.filename
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            processed_df = process_file(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            output_filename = filename.replace('.xlsx', '_processed.xlsx')
            output_path = os.path.join(app.config['OUTPUT_FOLDER'], output_filename)
            processed_df.to_excel(output_path, index=False)

    output_files = os.listdir(app.config['OUTPUT_FOLDER'])
    return render_template('index.html', output_files=output_files)

@app.route('/download/<filename>')
def download_file(filename):
    return send_file(os.path.join(app.config['OUTPUT_FOLDER'], filename), as_attachment=True)


if __name__ == '__main__':
    from pyngrok import ngrok

    public_url = ngrok.connect()
    print(' * Tunnel URL:', public_url)

    app.run(port=80)



 * Tunnel URL: NgrokTunnel: "https://9008-34-125-96-108.ngrok-free.app" -> "http://localhost:80"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:80
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Apr/2024 12:17:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2024 12:17:04] "GET /favicon.ico HTTP/1.1" 404 -
